In [5]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import os

os.chdir('/home/pp/Desktop/Pradyumn Pundir/MLOps/new_main/notebooks')

def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config['mlflow_config']
    model_name = mlflow_config['registered_model_name']
    remote_server_uri = mlflow_config['remote_server_uri']
    
    mlflow.set_registry_uri(remote_server_uri)

    runs = mlflow.search_runs(experiment_ids = 1) # 1 because in the artifacts we have the folder name as 1
    # 0 is for the default and there can be more than one depending upon the number of experiments we have

    lowest = runs['metrics.mae'].sort_values(ascending=True)
    lowest_run_id = runs[runs['metrics.mae'] == lowest]['run_id'][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv['run_id'] == lowest_run_id:
            current_version = mv['version']
            logged_model = mv['source']
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name = model_name,
                version=current_version,
                stage='Production'
            )
        else:
            current_version = mv['version']
            logged_model = mv['source']
            client.transition_model_version_stage(
                name = model_name,
                version=current_version,
                stage='Staging'
            )

log_production_model('../params.yaml')

TypeError: object of type 'int' has no len()

In [2]:
pwd

'/home/pp/Desktop/Pradyumn Pundir/MLOps/new_main/notebooks'